# Bài tập 2.2.4: Xây dựng mô hình SVM với Animal_condition_Predict

## 1. Import thư viện

In [35]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import svm
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV

## 2. Load dữ liệu

In [36]:
df = pd.read_csv('./data/Animal_Condition_Predict.csv')
features_response = df.columns.tolist()

In [37]:
df.head()

,AnimalName,symptoms1,symptoms2,symptoms3,symptoms4,symptoms5,Dangerous
0,Dog,Fever,Diarrhea,Vomiting,Weight loss,Dehydration,Yes
1,Dog,Fever,Diarrhea,Coughing,Tiredness,Pains,Yes
2,Dog,Fever,Diarrhea,Coughing,Vomiting,Anorexia,Yes
3,Dog,Fever,Difficulty breathing,Coughing,Lethargy,Sneezing,Yes
4,Dog,Fever,Diarrhea,Coughing,Lethargy,Blue Eye,Yes


In [38]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 871 entries, 0 to 870
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   AnimalName  871 non-null    object
 1   symptoms1   871 non-null    object
 2   symptoms2   871 non-null    object
 3   symptoms3   871 non-null    object
 4   symptoms4   871 non-null    object
 5   symptoms5   871 non-null    object
 6   Dangerous   869 non-null    object
dtypes: object(7)
memory usage: 47.8+ KB


In [39]:
df[df.duplicated()].shape

(28, 7)

In [40]:
df.isnull().sum()

AnimalName    0
symptoms1     0
symptoms2     0
symptoms3     0
symptoms4     0
symptoms5     0
Dangerous     2
dtype: int64

In [41]:
print("\n=== CHECKING DATA TYPES ===")
for col in df.columns:
    dtype = df[col].dtype
    unique_count = df[col].nunique()
    print(f"{col}: {dtype} ({unique_count} unique values)")



=== CHECKING DATA TYPES ===
AnimalName: object (46 unique values)
symptoms1: object (232 unique values)
symptoms2: object (230 unique values)
symptoms3: object (229 unique values)
symptoms4: object (217 unique values)
symptoms5: object (203 unique values)
Dangerous: object (2 unique values)


## 3. Làm sạch dữ liệu

In [42]:
df = df.drop_duplicates()

In [43]:
df = df.dropna()

## 4. Mã hóa dữ liệu

In [44]:
le = LabelEncoder()
df_encoder = df.copy()
for col in df.columns:
    if df[col].dtype == 'object':
        df_encoder[col] = le.fit_transform(df[col])
df_encoder.head()


,AnimalName,symptoms1,symptoms2,symptoms3,symptoms4,symptoms5,Dangerous
0,6,63,30,179,181,32,1
1,6,63,30,31,164,113,1
2,6,63,30,31,172,8,1
3,6,63,33,31,87,142,1
4,6,63,30,31,87,21,1


In [45]:
X = df_encoder.iloc[:, :-1]
y = df_encoder.iloc[:, -1]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=101)

## 5. Huấn luyện mô hình SVM

In [46]:
# khởi tạo mô hình phân lớp
clf = svm.SVC()
# Sử dụng phương thức 'fit' để huấn luyện mô hình với dữ liệu huấn luyện và nhãn huấn luyện
# fit (X,Y) với X là tập các đối tượng, Y là tập nhãn tương ứng của đối tượng.
clf.fit(X_train, y_train)

,C,1.0
,kernel,'rbf'
,degree,3
,gamma,'scale'
,coef0,0.0
,shrinking,True
,probability,False
,tol,0.001
,cache_size,200
,class_weight,None
,verbose,False


## 6. Đánh giá độ chính xác

In [47]:
# Tính độ chính xác trên tập huấn luyện và tập kiểm tra
train_acc = clf.score(X_train,y_train)
val_acc = clf.score(X_test,y_test)
print('Training accuracy: {}'.format(train_acc))
print('Validation accuracy: {}'.format(val_acc))

Training accuracy: 0.9806547619047619
Validation accuracy: 0.9585798816568047


## 7.Tìm tham số kernel 

In [48]:
# best_svm, best_val_acc và best_kernel lần lượt là các biến lưu mô hình tốt nhất,
# độ chính xác cao nhất trên tập kiểm tra và kernel tốt nhất
kernels = ['linear', 'poly', 'rbf', 'sigmoid']
best_svm = None
best_val_acc = -1
best_kernel = None
# Huấn luyện các mô hình dựa trên dữ liệu huấn luyện và tham số kernel
# Tính toán độ chính xác trên tập huấn luyện và tập kiểm tra để tìm được mô hình tốt nhất
for i in range(4):
    clf = svm.SVC(kernel=kernels[i], probability=True)
    clf.fit(X_train, y_train)
    tmp_val_acc = clf.score(X_test, y_test)
    if (tmp_val_acc > best_val_acc):
        best_val_acc = tmp_val_acc
        best_svm = clf
        best_kernel = kernels[i]
# Hiển thị mô hình tốt nhất cùng với độ chính xác
print("Best validation accuracy : {} with kernel: {}".format(best_val_acc,
best_kernel))

Best validation accuracy : 0.9585798816568047 with kernel: linear


Sử dựng GridSearchCV để tìm tham số tối ưu

In [49]:
params = {'kernel': ['linear', 'poly', 'rbf', 'sigmoid']}
clf = svm.SVC()
cv = GridSearchCV(clf, param_grid=params,
                 scoring='accuracy',
                 cv=5, verbose=1,
                 error_score=np.nan, n_jobs=None, return_train_score=True)
cv.fit(X_train, y_train)
print("Best parameters found: ", cv.best_params_)
print("Best cross-validation accuracy: ", cv.best_score_)

Fitting 5 folds for each of 4 candidates, totalling 20 fits
Best parameters found:  {'kernel': 'linear'}
Best cross-validation accuracy:  0.9806633499170812


# Kết thúc